In [1]:
import sys
sys.path.insert(0, '../../../network')
import numpy as np
from sequences import GaussianSequence
from connectivity import cmatrix
from transfer_functions import ErrorFunction
from network import Population, RateNetwork

### Population and patterns

In [2]:
# Pop size
N_ctx, N_d1 = 3000, 3000
N = [N_ctx, N_d1]

# Gaussian patterns
S, P = 1, 4
sequences_ctx = [GaussianSequence(P,N_ctx, seed=873) for i in range(S)]
sequences_d1 = [GaussianSequence(P,N_d1, seed=11) for i in range(S)]
sequences = np.array([sequences_ctx, sequences_d1], dtype='object')

### Synaptic connectivity rule

In [3]:
# Learning rule: 
# Type 0: zeros; 1: symmetric; 2: asymmetric
A00, A01 = cmatrix((P,P), 4, 1), cmatrix((P,P), 6, 1)
A0 = np.array([A00,A01])


A10, A11, = cmatrix((P,P), .15, 1), cmatrix((P,P), 4, 1)
A1 = np.array([A10,A11])


A = np.array([A0,A1])

In [4]:
# Connection probabilities
cp = np.array([[0.1, 0.1],
               [0.1, 0.1]])

cw = np.array([[0,1],
                [0,-1]])

In [5]:
np.savez('./params.npz', 
         N=N, sequences=sequences,
         cp=cp, cw=cw, A=A)